In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import os
import glob
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import layers
from keras import optimizers
from keras import Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
# from keras import Activation
from keras.models import Model, Sequential

In [4]:
labels = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

col = {'c0': 'safe driving',
'c1': 'texting - right',
'c2': 'talking on the phone - right',
'c3': 'texting - left',
'c4': 'talking on the phone - left',
'c5':'operating the radio',
'c6': 'drinking',
'c7': 'reaching behind',
'c8': 'hair and makeup',
'c9': 'talking to passenger'}

In [5]:
DATA_PATH = "/kaggle/input/state-farm-distracted-driver-detection/imgs"
train_path = os.path.join(DATA_PATH, "train")
test_path = os.path.join(DATA_PATH, "test")

In [6]:
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [7]:
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.3, zoom_range=0.3,
                                   channel_shift_range=0.0,
                                   fill_mode='nearest', cval=0.0, horizontal_flip=True, vertical_flip=False, rescale=1/255.,
                                   data_format='channels_last', validation_split=0.3,
                                   dtype='float32')

In [8]:
val_datagen = ImageDataGenerator(rescale=1/255.,
                                   data_format='channels_last', validation_split=0.3,
                                   dtype='float32')

In [9]:
train_generator = train_datagen.flow_from_directory(train_path, target_size=(256,256), color_mode="grayscale", 
                                                    class_mode="categorical", batch_size=64, subset="training")

Found 15702 images belonging to 10 classes.


In [10]:
val_generator = val_datagen.flow_from_directory(train_path, target_size=(256,256), color_mode="grayscale", 
                                                    class_mode="categorical", batch_size=64, subset="validation")

Found 6722 images belonging to 10 classes.


In [11]:
df = pd.read_csv("/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv")

In [12]:
def make_model(input_shape=(256,256,1)):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=input_shape, activation='elu'))
#     model.add(Activation(activation='elu'))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='elu'))
#     model.add(Activation(activation=elu))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='elu'))
#     model.add(Activation(activation=elu))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='elu'))
#     model.add(Activation(activation=elu))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())

    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='elu'))
#     model.add(Activation(activation=elu))
    model.add(MaxPooling2D())
    model.add(BatchNormalization())

#     model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='elu'))
# #     model.add(Activation(activation=elu))
#     model.add(MaxPooling2D())
#     model.add(BatchNormalization())

    # model.add(Conv2D(filters=32, kernel_size=(3, 3), activation=elu))
    # model.add(Activation(activation=elu))
    # model.add(MaxPooling2D())
    # model.add(BatchNormalization())

    model.add(GlobalAveragePooling2D())
    model.add(Dense(3000, activation='elu'))
#     model.add(Activation(activation=elu))
    model.add(Dropout(rate=0.25))
    model.add(Dense(2000, activation='elu'))
#     model.add(Activation(activation='elu'))
    model.add(Dropout(rate=0.25))
    model.add(Dense(10, activation='softmax'))
    
    return (model)
    

In [13]:
model = make_model()
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])    

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 127, 127, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 62, 62, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 64)       

In [15]:
epochs = 20
batch_size = 64

In [16]:
filepath = 'distracted_driver.h5'
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5,
                             verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)
callback_l = [reducelr, checkpoint]

In [17]:
history = model.fit_generator(train_generator,
                              steps_per_epoch= train_generator.samples // batch_size,
                              epochs=epochs,
                              validation_data=val_generator,
                              validation_steps=val_generator.samples // batch_size)

Epoch 1/20
245/245 [==============================] - 229s 933ms/step - loss: 2.3323 - acc: 0.1906 - val_loss: 3.6649 - val_acc: 0.0851
Epoch 2/20
245/245 [==============================] - 189s 773ms/step - loss: 1.9372 - acc: 0.3017 - val_loss: 2.3706 - val_acc: 0.2212
Epoch 3/20
245/245 [==============================] - 190s 774ms/step - loss: 1.4670 - acc: 0.4886 - val_loss: 2.3745 - val_acc: 0.2915
Epoch 4/20
245/245 [==============================] - 188s 769ms/step - loss: 0.9318 - acc: 0.6780 - val_loss: 0.5834 - val_acc: 0.7044
Epoch 5/20
245/245 [==============================] - 190s 775ms/step - loss: 0.6748 - acc: 0.7746 - val_loss: 1.4666 - val_acc: 0.6793
Epoch 6/20
245/245 [==============================] - 187s 765ms/step - loss: 0.5558 - acc: 0.8165 - val_loss: 4.4796 - val_acc: 0.3480
Epoch 7/20
245/245 [==============================] - 186s 758ms/step - loss: 0.4569 - acc: 0.8468 - val_loss: 0.3582 - val_acc: 0.9160
Epoch 8/20
245/245 [============================

In [18]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
print("Saved model to disk")

Saved model to disk
